In [1]:
cd ../../../datasets/190829_Kanshan_zjfx

F:\Codes\datasets\190829_Kanshan_zjfx


In [2]:
ls

 驱动器 F 中的卷是 存储
 卷的序列号是 C14D-581B

 F:\Codes\datasets\190829_Kanshan_zjfx 的目录

2019/11/19  08:48    <DIR>          .
2019/11/19  08:48    <DIR>          ..
2019/09/26  19:08     5,805,965,112 answer_info_0926.txt
2019/11/08  14:33               188 assert.py
2019/09/26  18:10       333,836,065 invite_info_0926.txt
2019/09/28  12:14        37,860,903 invite_info_evaluate_1_0926.txt
2019/09/26  17:12       552,574,982 member_info_0926.txt
2019/11/08  21:28     1,144,346,559 question_info_0926.sql
2019/09/26  18:01     1,074,273,891 question_info_0926.txt
2019/10/18  16:35    <DIR>          sample
2019/08/05  17:19        15,951,125 single_word_vectors_64d.txt
2019/08/05  17:20        69,411,196 topic_vectors_64d.txt
2019/11/18  18:20     4,006,275,165 train.csv
2019/11/18  17:11     3,456,569,215 train.sql
2019/11/18  17:38       484,214,123 val.csv
2019/08/05  17:19     1,230,810,962 word_vectors_64d.txt
2019/11/06  09:59     9,418,974,739 zhihu2019_dataset.sql
              14 个文件 27,63

## 0.1 超参数

In [3]:
import numpy as np
import pandas as pd
# import pymysql
import torch
from tqdm import tqdm

BATCH_SIZE = 65536
DB_NAME = 'zhihu2019_dataset'
DO_DATASET_BALANCE = True
EPOCH = 80
LR = 0.1
TRAIN_ALL = True

PKL_DIR_READ = '../../jupyter/190919_DataMiningHW/project3/storage/train_param.pkl'
PKL_DIR_OUT = '../../jupyter/190919_DataMiningHW/project3/storage/train_param.pkl'
RESULT_DIR_OUT = '../../jupyter/190919_DataMiningHW/project3/storage/result.txt'

DEVICE = torch.device('cuda:3') if torch.cuda.is_available() else torch.device('cpu')
DEVICE

device(type='cpu')

## 1.1 加载数据集

取训练集。

In [4]:
# with DB() as db:
#     db.execute('select * from train')
#     train = pd.DataFrame(db.fetchall())

train = pd.read_csv('train.csv', header=0, sep=',')

取测试集。

In [5]:
# with DB() as db:
#     db.execute('select * from val')
#     test = pd.DataFrame(db.fetchall())

test = pd.read_csv('val.csv', header=0, sep=',')

合并二集。

In [6]:
col_list = ['q_id', 'u_id', 'do_answer', 'invite_hour', 'follow_topic_hit', 
             'interest_topic_hit', 'sex', 'access_freq', 'bin_feat_a', 'bin_feat_b', 
             'bin_feat_c', 'bin_feat_d', 'bin_feat_e', 'multi_feat_a', 'multi_feat_b',
             'multi_feat_c', 'multi_feat_d', 'multi_feat_e', 'salt', 'u_answer', 
             'u_good_answer', 'u_recommand_answer', 'u_image_answer', 'u_video_answer', 
             'u_word_avg', 'u_like_avg', 'u_unlike_avg', 'u_comment_avg', 
             'u_collect_avg', 'u_thanks_avg', 'u_report_avg','u_nohelp_avg', 
             'u_oppose_avg', 'q_answer', 'q_good_answer', 'q_recommand_answer',
             'q_image_answer', 'q_video_answer', 'q_word_avg', 'q_like', 'q_comment', 
             'q_collect', 'q_thanks']
# train = train[col_list]
# test = test[col_list]

In [7]:
data = pd.concat([train, test], axis=0)
data.columns

Index(['q_id', 'u_id', 'do_answer', 'invite_hour', 'follow_topic_hit',
       'interest_topic_hit', 'sex', 'access_freq', 'bin_feat_a', 'bin_feat_b',
       'bin_feat_c', 'bin_feat_d', 'bin_feat_e', 'multi_feat_a',
       'multi_feat_b', 'multi_feat_c', 'multi_feat_d', 'multi_feat_e', 'salt',
       'u_answer', 'u_good_answer', 'u_recommand_answer', 'u_image_answer',
       'u_video_answer', 'u_word_avg', 'u_like_avg', 'u_unlike_avg',
       'u_comment_avg', 'u_collect_avg', 'u_thanks_avg', 'u_report_avg',
       'u_nohelp_avg', 'u_oppose_avg', 'q_answer', 'q_good_answer',
       'q_recommand_answer', 'q_image_answer', 'q_video_answer', 'q_word_avg',
       'q_like', 'q_comment', 'q_collect', 'q_thanks'],
      dtype='object')

标准化数据

In [8]:
from sklearn.preprocessing import scale

for i in ['invite_hour', 'follow_topic_hit', 
             'interest_topic_hit', 'bin_feat_a', 'bin_feat_b', 
             'bin_feat_c', 'bin_feat_d', 'bin_feat_e', 'salt', 'u_answer', 
             'u_good_answer', 'u_recommand_answer', 'u_image_answer', 'u_video_answer', 
             'u_word_avg', 'u_like_avg', 'u_unlike_avg', 'u_comment_avg', 
             'u_collect_avg', 'u_thanks_avg', 'u_report_avg','u_nohelp_avg', 
             'u_oppose_avg', 'q_answer', 'q_good_answer', 'q_recommand_answer',
             'q_image_answer', 'q_video_answer', 'q_word_avg', 'q_like', 'q_comment', 
             'q_collect', 'q_thanks']:
    data[i] = scale(data[i].values)

## 1.1.5 清洗准备工作

计算数据相关性

In [9]:
for i in ['invite_hour', 'follow_topic_hit', 
             'interest_topic_hit', 'bin_feat_a', 'bin_feat_b', 
             'bin_feat_c', 'bin_feat_d', 'bin_feat_e', 'salt', 'u_answer', 
             'u_good_answer', 'u_recommand_answer', 'u_image_answer', 'u_video_answer', 
             'u_word_avg', 'u_like_avg', 'u_unlike_avg', 'u_comment_avg', 
             'u_collect_avg', 'u_thanks_avg', 'u_report_avg','u_nohelp_avg', 
             'u_oppose_avg', 'q_answer', 'q_good_answer', 'q_recommand_answer',
             'q_image_answer', 'q_video_answer', 'q_word_avg', 'q_like', 'q_comment', 
             'q_collect', 'q_thanks']:
    k1 = data['do_answer'].corr(data[i], method='pearson')
    k2 = data['do_answer'].corr(data[i], method='spearman')
    k3 = data['do_answer'].corr(data[i], method='kendall')
    print('%s\t\t%.4f\t%.4f\t%.4f' % (i, k1, k2, k3))

invite_hour		-0.0319	-0.0310	-0.0253
follow_topic_hit		0.0297	0.0265	0.0252
interest_topic_hit		0.0078	0.0080	0.0075
bin_feat_a		0.0147	0.0145	0.0141
bin_feat_b		-0.0117	-0.0123	-0.0120
bin_feat_c		-0.0082	-0.0085	-0.0082
bin_feat_d		0.0012	0.0010	0.0010
bin_feat_e		-0.0000	-0.0001	-0.0001
salt		0.0404	0.0526	0.0417
u_answer		0.1213	0.1572	0.1295
u_good_answer		0.0025	0.0039	0.0038
u_recommand_answer		0.0003	0.0002	0.0002
u_image_answer		-0.0121	0.0209	0.0190
u_video_answer		-0.0065	0.0101	0.0097
u_word_avg		-0.0024	0.0629	0.0509
u_like_avg		-0.0063	-0.0116	-0.0106
u_unlike_avg		-0.0075	-0.0163	-0.0156
u_comment_avg		-0.0083	0.0073	0.0067
u_collect_avg		-0.0028	-0.0118	-0.0112
u_thanks_avg		-0.0039	-0.0196	-0.0187
u_report_avg		-0.0005	-0.0004	-0.0004
u_nohelp_avg		-0.0045	-0.0089	-0.0086
u_oppose_avg		-0.0075	-0.0163	-0.0156
q_answer		0.0867	0.3735	0.3184
q_good_answer		0.0238	0.0249	0.0241
q_recommand_answer		0.0026	0.0029	0.0028
q_image_answer		0.0600	0.1606	0.1499
q_video_answer		0

## 1.2 处理数据集

清洗数据

In [10]:
stable_col_list = ['q_id', 'u_id', 'do_answer']
good_col_list = ['follow_topic_hit', 'interest_topic_hit', 'bin_feat_a',
                 'salt', 'u_answer', 'u_image_answer', 'u_video_answer', 
                 'u_word_avg', 'q_answer', 'q_good_answer',
                 'q_image_answer', 'q_video_answer', 'q_word_avg', 'q_like', 'q_comment', 
                 'q_collect', 'q_thanks']
good_bad_col_list = ['invite_hour', 'follow_topic_hit', 
             'interest_topic_hit', 'bin_feat_a', 'bin_feat_b', 
             'bin_feat_c', 'bin_feat_d', 'bin_feat_e', 'salt', 'u_answer', 
             'u_good_answer', 'u_recommand_answer', 'u_image_answer', 'u_video_answer', 
             'u_word_avg', 'u_like_avg', 'u_unlike_avg', 'u_comment_avg', 
             'u_collect_avg', 'u_thanks_avg', 'u_report_avg','u_nohelp_avg', 
             'u_oppose_avg', 'q_answer', 'q_good_answer', 'q_recommand_answer',
             'q_image_answer', 'q_video_answer', 'q_word_avg', 'q_like', 'q_comment', 
             'q_collect', 'q_thanks']
other_col_list = ['sex', 'access_freq', 'multi_feat_a', 'multi_feat_b', 
                  'multi_feat_c','multi_feat_d','multi_feat_e']
data = data[stable_col_list + good_bad_col_list + other_col_list]
data.columns

Index(['q_id', 'u_id', 'do_answer', 'invite_hour', 'follow_topic_hit',
       'interest_topic_hit', 'bin_feat_a', 'bin_feat_b', 'bin_feat_c',
       'bin_feat_d', 'bin_feat_e', 'salt', 'u_answer', 'u_good_answer',
       'u_recommand_answer', 'u_image_answer', 'u_video_answer', 'u_word_avg',
       'u_like_avg', 'u_unlike_avg', 'u_comment_avg', 'u_collect_avg',
       'u_thanks_avg', 'u_report_avg', 'u_nohelp_avg', 'u_oppose_avg',
       'q_answer', 'q_good_answer', 'q_recommand_answer', 'q_image_answer',
       'q_video_answer', 'q_word_avg', 'q_like', 'q_comment', 'q_collect',
       'q_thanks', 'sex', 'access_freq', 'multi_feat_a', 'multi_feat_b',
       'multi_feat_c', 'multi_feat_d', 'multi_feat_e'],
      dtype='object')

离散特征的`LabelEncoder`编码。

In [11]:
from sklearn.preprocessing import LabelEncoder

# 离散值编码
encoder = LabelEncoder()
for i in other_col_list:
    data[i] = encoder.fit_transform(data[i])
    
data.iloc[:10]

,q_id,u_id,do_answer,invite_hour,follow_topic_hit,interest_topic_hit,bin_feat_a,bin_feat_b,bin_feat_c,bin_feat_d,...,q_comment,q_collect,q_thanks,sex,access_freq,multi_feat_a,multi_feat_b,multi_feat_c,multi_feat_d,multi_feat_e
0,Q2166419046,M401693808,0,1.546644,1.313413,-0.274834,-1.404298,1.532982,-0.201699,-0.591582,...,-0.074245,-0.028820,-0.044241,2,4,1917,170,244,799,1
1,Q1550017551,M3392373099,0,-0.389192,-0.438017,-0.274834,0.712100,-0.652323,-0.201699,-0.591582,...,-0.070538,-0.028820,-0.044241,2,1,1092,115,249,657,0
2,Q604029601,M2317670257,0,0.314748,-0.438017,-0.274834,0.712100,-0.652323,-0.201699,-0.591582,...,-0.074245,-0.028820,-0.044241,2,4,1380,206,0,440,1
3,Q2350061229,M1618461867,0,-0.389192,-0.438017,2.528689,0.712100,-0.652323,-0.201699,1.690382,...,-0.063124,-0.028304,-0.041673,2,0,1092,115,244,799,1
4,Q2443223942,M3544409350,0,-1.621091,-0.438017,-0.274834,0.712100,-0.652323,-0.201699,-0.591582,...,-0.066831,-0.028820,-0.044241,2,1,506,204,175,705,1
5,Q640765464,M2818659842,0,0.490733,1.313413,-0.274834,0.712100,-0.652323,-0.201699,-0.591582,...,-0.074245,-0.028820,-0.044241,1,0,1380,206,0,440,1
6,Q795459266,M2818659842,0,1.194672,-0.438017,-0.274834,0.712100,-0.652323,-0.201699,-0.591582,...,-0.055711,-0.028820,-0.044241,1,0,1380,206,0,440,1
7,Q190554387,M1581217469,1,-0.917147,-0.438017,-0.274834,0.712100,-0.652323,-0.201699,-0.591582,...,-0.066831,-0.028820,-0.041673,2,1,1092,115,0,128,1
8,Q1958712851,M3021021791,0,1.018691,-0.438017,-0.274834,-1.404298,1.532982,-0.201699,-0.591582,...,-0.074245,-0.028820,-0.044241,2,4,1486,170,249,657,1
9,Q311993584,M1766315480,0,0.314748,-0.438017,-0.274834,0.712100,-0.652323,-0.201699,-0.591582,...,-0.048297,-0.028304,-0.036537,1,0,1092,115,370,1252,1


## 1.3 包装数据集

In [12]:
# import numpy as np

# if DO_DATASET_BALANCE:
#     temp = data[data['do_answer'] == 1]  # 用于均衡的数据
#     print('用于均衡的数据有 %d 条.' % len(temp))
#     train_x = data[data['do_answer']>-1].append([temp, temp, temp])
# else:
#     train_x = data[data['do_answer']>-1]
# train_x, train_y = train_x[col_list[3:]].values, train_x[['do_answer']].values[:, 0]

test_x = data[data['do_answer']==-1]
test_x, test_y = test_x[good_bad_col_list + other_col_list].values, test_x['do_answer'].values
# print('ok.')

In [13]:
# 正负样本df
positive_samples = data[data['do_answer'] == 1]
negative_samples = data[data['do_answer'] == 0]
print('pos_len:', len(positive_samples))
print('neg_len:', len(negative_samples))

pos_len: 1682914
neg_len: 7806219


In [14]:
from torch.utils.data import Dataset, DataLoader


class MyDataset(Dataset):
    """每次new都会随机均衡数据集。"""
    def __init__(self, mode='train'):
        self.mode = mode
        if mode == 'train':
            train_x = pd.concat([
                positive_samples,
                negative_samples.sample(n=len(positive_samples), axis=0)
            ], axis=0)
            train_x, train_y = train_x[good_bad_col_list + other_col_list].values, train_x['do_answer'].values
            self.data = {
                'x': train_x,
                'y': train_y,
            }
        elif mode == 'test':
            self.data = {
                'x': test_x,
                'y': test_y,
            }
        else:
            raise
            
    def __getitem__(self, index):
        return self.data['x'][index], self.data['y'][index]
        
    def __len__(self):
        return len(self.data['x'])
    
    
print('ok.')

ok.


In [15]:
import math
from torch.utils.data import random_split

train_size, val_size, test_size = 0, 0, 0
train_len, val_len, test_len = 0, 0, 0
data_loader = {}

def flush_dataloader(log=True):
    global train_size, val_size, test_size, train_len, val_len, test_len, data_loader
    
    full_train_dataset = MyDataset(mode='train')
    test_dataset = MyDataset(mode='test')

    if TRAIN_ALL:
        data_loader = {
            'train': DataLoader(dataset=full_train_dataset, 
                                batch_size=BATCH_SIZE, 
#                                 num_workers=5,
                                shuffle=True),
            'test': DataLoader(dataset=test_dataset,
                               batch_size=BATCH_SIZE,
#                                num_workers=5,
                               shuffle=False),
        }
        train_size = len(full_train_dataset)
        test_size = len(test_dataset)
    else:
        train_size = int(0.8 * len(full_train_dataset))
        val_size = len(full_train_dataset) - train_size
        train_dataset, val_dataset = random_split(full_train_dataset, [train_size, val_size])

        data_loader = {
            'train': DataLoader(dataset=train_dataset, 
                                batch_size=BATCH_SIZE, 
                               num_workers=5,
                                shuffle=True),
            'val': DataLoader(dataset=val_dataset, 
                              batch_size=BATCH_SIZE,
                               num_workers=5,
                              shuffle=False),
            'test': DataLoader(dataset=test_dataset,
                               batch_size=BATCH_SIZE,
                               num_workers=5,
                               shuffle=False),
        }
        train_size = len(train_dataset)
        val_size = len(val_dataset)
        test_size = len(test_dataset)

    if log:
        print('train_size:', train_size)
        print('val_size:', val_size)
        print('test_size:', test_size)

    # 计算一轮enumerate的长度
    train_len = math.ceil(train_size / BATCH_SIZE)
    val_len = math.ceil(val_size / BATCH_SIZE)
    test_len = math.ceil(test_size / BATCH_SIZE)

    if log:
        print('train_len:', train_len)
        print('val_len:', val_len)
        print('test_len:', test_len)

## 2.1 构建网络

In [16]:
import torch
import torch.nn.functional as F
from torch import nn


class MyNet_v2(nn.Module):
    def __init__(self, in_dim, out_dim=2, norm='dropout'):
        super(MyNet_v2, self).__init__()
        self.norm = norm

        in_dim = in_dim+1+1+4+2+2+3
        self.embed_sex = nn.Embedding(3, 2)
        self.embed_access_freq = nn.Embedding(5, 2)
        self.embed_multi_a = nn.Embedding(2561, 5)
        self.embed_multi_b = nn.Embedding(291, 3)
        self.embed_multi_c = nn.Embedding(428, 3)
        self.embed_multi_d = nn.Embedding(1556, 4)
        self.embed_multi_e = nn.Embedding(2, 1)
        self.fc1 = nn.Linear(in_dim, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, out_dim)
        self.relu = nn.ReLU(inplace=True)
        if self.norm == 'batchnorm':
            self.bn0 = nn.BatchNorm1d(in_dim)
            self.bn1 = nn.BatchNorm1d(512)
            self.bn2 = nn.BatchNorm1d(256)
            self.bn3 = nn.BatchNorm1d(128)


    def forward(self, x):
        sex = self.embed_sex(x[:, -7].long())
        access_freq = self.embed_access_freq(x[:, -6].long())
        multi_a = self.embed_multi_a(x[:, -5].long())
        multi_b = self.embed_multi_b(x[:, -4].long())
        multi_c = self.embed_multi_c(x[:, -3].long())
        multi_d = self.embed_multi_d(x[:, -2].long())
        multi_e = self.embed_multi_e(x[:, -1].long())

        out = x[:, :-7]
        out = torch.cat(
            (out, sex, access_freq, multi_a, multi_b, multi_c, multi_d, multi_e), dim=1)
        
        if self.norm is None:
            out = self.fc1(out)
            out = self.fc2(self.relu(out))
            out = self.fc3(self.relu(out))
            out = self.fc4(self.relu(out))
        elif self.norm == 'dropout':
            out = self.fc1(out)
            out = F.dropout(out, p=0.5, training=self.training)
            out = self.fc2(self.relu(out))
            out = F.dropout(out, p=0.5, training=self.training)
            out = self.fc3(self.relu(out))
            out = F.dropout(out, p=0.5, training=self.training)
            out = self.fc4(self.relu(out))
        else:
            out = self.fc1(self.bn0(out))
            out = self.fc2(self.relu(self.bn1(out)))
            out = self.fc3(self.relu(self.bn2(out)))
            out = self.fc4(self.relu(self.bn3(out)))
        return out

In [17]:
# from my_model import MyNet_v2_1

net = MyNet_v2(len(good_bad_col_list + other_col_list)).to(DEVICE)
net

MyNet_v2(
  (embed_sex): Embedding(3, 2)
  (embed_access_freq): Embedding(5, 2)
  (embed_multi_a): Embedding(2561, 5)
  (embed_multi_b): Embedding(291, 3)
  (embed_multi_c): Embedding(428, 3)
  (embed_multi_d): Embedding(1556, 4)
  (embed_multi_e): Embedding(2, 1)
  (fc1): Linear(in_features=53, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=128, bias=True)
  (fc4): Linear(in_features=128, out_features=2, bias=True)
  (relu): ReLU(inplace)
)

In [17]:
# 读取现有的
if torch.cuda.is_available():
    net.load_state_dict(torch.load(PKL_DIR_READ))
else:
    net.load_state_dict(torch.load(PKL_DIR_READ, map_location='cpu'))

## 2.2 优化器、损失函数

In [18]:
EPOCH = 100
LR = 0.1

In [19]:
from torch import optim, nn

loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=LR, momentum=0.9)
# scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.75)  # 步进
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max',verbose=1,patience=3)
# scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max = (EPOCH // 9) + 1)

'ok'

'ok'

## 2.3 训练

In [ ]:
from tqdm import tqdm

for epoch in range(EPOCH):
    count, running_correct, running_loss = 0, 0, 0
    flush_dataloader(log=False)
    
    net.train()
    with tqdm(total=train_len) as pbar:
        for step, (bx, by) in enumerate(data_loader['train']):
            # 训练
            bx = bx.float().to(DEVICE)
            by = by.long().to(DEVICE)

            prediction = net(bx)
            loss = loss_function(prediction, by)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # 统计成效
            pre = torch.argmax(torch.softmax(prediction, dim=1), dim=1)
            my_loss = float(loss.data.cpu().numpy())
            my_correct = sum(pre.cpu().numpy() == by.cpu().numpy())
            my_acc = my_correct / BATCH_SIZE
            count += 1
            running_loss += my_loss
            running_correct += my_correct
            
            # 更新进度条
            pbar.update(1)
            pbar.set_description('Epoch: %d, Loss: %.4f, Acc: %.4f, Lr: %.5f' % (
                                   epoch,
                                   running_loss / count,
                                   running_correct / train_size,
                                   optimizer.state_dict()['param_groups'][0]['lr']))
    
    # 输出一轮结果
#     print('Epoch: %d, Loss: %.4f, Acc: %.4f, Lr: %.5f' % (
#            epoch,
#            running_loss / count,
#            running_correct / train_size,
#            optimizer.state_dict()['param_groups'][0]['lr']))
    
    # 保存参数
    torch.save(net.state_dict(), PKL_DIR_OUT)
    
    # lr scheduler
#     scheduler.step()
    scheduler.step(running_correct / train_size)

Epoch: 9, Loss: 0.5150, Acc: 0.7393, Lr: 0.10000: 100%|██████████| 52/52 [01:57<00:00,  1.83s/it]
Epoch: 10, Loss: 0.5148, Acc: 0.7394, Lr: 0.10000: 100%|██████████| 52/52 [01:56<00:00,  1.83s/it]
Epoch: 11, Loss: 0.5149, Acc: 0.7394, Lr: 0.10000: 100%|██████████| 52/52 [01:57<00:00,  1.89s/it]
Epoch: 12, Loss: 0.5146, Acc: 0.7394, Lr: 0.10000: 100%|██████████| 52/52 [01:57<00:00,  1.83s/it]
Epoch: 13, Loss: 0.5146, Acc: 0.7397, Lr: 0.10000: 100%|██████████| 52/52 [01:58<00:00,  1.82s/it]
Epoch: 14, Loss: 0.5143, Acc: 0.7400, Lr: 0.10000: 100%|██████████| 52/52 [01:59<00:00,  1.88s/it]
Epoch: 15, Loss: 0.5139, Acc: 0.7402, Lr: 0.10000: 100%|██████████| 52/52 [02:00<00:00,  1.85s/it]
Epoch: 16, Loss: 0.5139, Acc: 0.7399, Lr: 0.10000: 100%|██████████| 52/52 [02:01<00:00,  1.96s/it]
Epoch: 17, Loss: 0.5139, Acc: 0.7400, Lr: 0.10000: 100%|██████████| 52/52 [01:58<00:00,  1.85s/it]
Epoch: 18, Loss: 0.5137, Acc: 0.7402, Lr: 0.10000: 100%|██████████| 52/52 [01:59<00:00,  1.91s/it]
Epoch: 19, 

Epoch    35: reducing learning rate of group 0 to 1.0000e-02.


Epoch: 36, Loss: 0.5098, Acc: 0.7431, Lr: 0.01000: 100%|██████████| 52/52 [01:58<00:00,  1.89s/it]
Epoch: 37, Loss: 0.5092, Acc: 0.7432, Lr: 0.01000: 100%|██████████| 52/52 [01:58<00:00,  1.84s/it]
Epoch: 38, Loss: 0.5087, Acc: 0.7435, Lr: 0.01000: 100%|██████████| 52/52 [01:57<00:00,  1.85s/it]
Epoch: 39, Loss: 0.5088, Acc: 0.7435, Lr: 0.01000: 100%|██████████| 52/52 [02:01<00:00,  1.97s/it]
Epoch: 40, Loss: 0.5089, Acc: 0.7434, Lr: 0.01000: 100%|██████████| 52/52 [02:01<00:00,  1.85s/it]
Epoch: 41, Loss: 0.5084, Acc: 0.7439, Lr: 0.01000: 100%|██████████| 52/52 [01:57<00:00,  1.84s/it]
Epoch: 42, Loss: 0.5087, Acc: 0.7436, Lr: 0.01000: 100%|██████████| 52/52 [01:59<00:00,  1.93s/it]
Epoch: 43, Loss: 0.5086, Acc: 0.7435, Lr: 0.01000: 100%|██████████| 52/52 [01:59<00:00,  1.95s/it]
Epoch: 44, Loss: 0.5087, Acc: 0.7434, Lr: 0.01000: 100%|██████████| 52/52 [01:58<00:00,  1.84s/it]
Epoch: 45, Loss: 0.5082, Acc: 0.7440, Lr: 0.01000: 100%|██████████| 52/52 [02:00<00:00,  1.87s/it]
Epoch: 46,

Epoch    49: reducing learning rate of group 0 to 1.0000e-03.


Epoch: 50, Loss: 0.5084, Acc: 0.7439, Lr: 0.00100: 100%|██████████| 52/52 [01:57<00:00,  1.86s/it]
Epoch: 51, Loss: 0.5087, Acc: 0.7436, Lr: 0.00100: 100%|██████████| 52/52 [01:58<00:00,  1.88s/it]
Epoch: 52, Loss: 0.5082, Acc: 0.7442, Lr: 0.00100: 100%|██████████| 52/52 [01:58<00:00,  1.85s/it]
Epoch: 53, Loss: 0.5081, Acc: 0.7441, Lr: 0.00100: 100%|██████████| 52/52 [01:58<00:00,  1.84s/it]
Epoch: 54, Loss: 0.5084, Acc: 0.7440, Lr: 0.00100: 100%|██████████| 52/52 [01:59<00:00,  1.92s/it]
Epoch: 55, Loss: 0.5082, Acc: 0.7440, Lr: 0.00100: 100%|██████████| 52/52 [01:58<00:00,  1.84s/it]
Epoch: 56, Loss: 0.5081, Acc: 0.7440, Lr: 0.00100: 100%|██████████| 52/52 [02:00<00:00,  1.86s/it]


Epoch    56: reducing learning rate of group 0 to 1.0000e-04.


Epoch: 57, Loss: 0.5082, Acc: 0.7441, Lr: 0.00010: 100%|██████████| 52/52 [01:57<00:00,  1.88s/it]
Epoch: 58, Loss: 0.5079, Acc: 0.7442, Lr: 0.00010: 100%|██████████| 52/52 [01:57<00:00,  1.83s/it]
Epoch: 59, Loss: 0.5079, Acc: 0.7443, Lr: 0.00010: 100%|██████████| 52/52 [01:58<00:00,  1.82s/it]
Epoch: 60, Loss: 0.5081, Acc: 0.7442, Lr: 0.00010: 100%|██████████| 52/52 [01:58<00:00,  1.89s/it]
Epoch: 61, Loss: 0.5080, Acc: 0.7443, Lr: 0.00010: 100%|██████████| 52/52 [01:57<00:00,  1.84s/it]
Epoch: 62, Loss: 0.5079, Acc: 0.7444, Lr: 0.00010: 100%|██████████| 52/52 [01:58<00:00,  1.87s/it]
Epoch: 63, Loss: 0.5081, Acc: 0.7442, Lr: 0.00010: 100%|██████████| 52/52 [01:58<00:00,  1.86s/it]
Epoch: 64, Loss: 0.5079, Acc: 0.7443, Lr: 0.00010: 100%|██████████| 52/52 [01:56<00:00,  1.83s/it]
Epoch: 65, Loss: 0.5083, Acc: 0.5505, Lr: 0.00010:  73%|███████▎  | 38/52 [01:26<00:31,  2.26s/it]

## 2.4 存储网络

In [ ]:
torch.save(net.state_dict(), PKL_DIR_OUT)

## 4.1 导出 test 结果

In [18]:
flush_dataloader(log=False)
result = data[data['do_answer'] == -1]
result.iloc[-1]

q_id                  Q2590286630
u_id                  M3168697168
do_answer                      -1
invite_hour             -0.741162
follow_topic_hit        -0.438017
interest_topic_hit      -0.274834
bin_feat_a                 0.7121
bin_feat_b              -0.652323
bin_feat_c              -0.201699
bin_feat_d              -0.591582
bin_feat_e              -0.283857
salt                      1.73948
u_answer                 -0.27225
u_good_answer          -0.0516322
u_recommand_answer    -0.00862002
u_image_answer            1.61585
u_video_answer          -0.103026
u_word_avg                1.47854
u_like_avg               0.134793
u_unlike_avg              1.01912
u_comment_avg             1.51127
u_collect_avg           0.0976203
u_thanks_avg             0.174823
u_report_avg           -0.0378529
u_nohelp_avg           -0.0296287
u_oppose_avg              1.01912
q_answer                -0.204937
q_good_answer          -0.0647244
q_recommand_answer     -0.0177534
q_image_answer

In [28]:
from tqdm import tqdm
# from my_model import MyNet_v2_2

net = MyNet_v2(len(good_bad_col_list + other_col_list)).to(DEVICE)
if torch.cuda.is_available():
    net.load_state_dict(torch.load(PKL_DIR_READ))
else:
    net.load_state_dict(torch.load(PKL_DIR_READ, map_location='cpu'))
net.eval()

res_list = []
with tqdm(total=test_len) as pbar:
    for step, (bx, by) in enumerate(data_loader['test']):
        bx = bx.float().to(DEVICE)
        by = by.long().to(DEVICE)
        
        prediction = net(bx)
        
        # 计算结果
        pre = torch.softmax(prediction, dim=1)[:, 1]
#         print(pre.cpu().detach().numpy())
#         raise
        res_list.extend(pre.cpu().detach().numpy())
        pbar.update(1)
        
result.loc[:, 'do_answer'] = res_list

result.iloc[:10]

100%|██████████| 18/18 [00:13<00:00,  1.53it/s]
C:\RAYIOOO\Programmings\Anaconda\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,q_id,u_id,do_answer,invite_hour,follow_topic_hit,interest_topic_hit,bin_feat_a,bin_feat_b,bin_feat_c,bin_feat_d,...,q_comment,q_collect,q_thanks,sex,access_freq,multi_feat_a,multi_feat_b,multi_feat_c,multi_feat_d,multi_feat_e
0,Q1493039281,M64135255,0.056410,-0.741162,-0.438017,1.373489,-1.404298,1.532982,-0.201699,-0.591582,...,-0.074245,-0.028820,-0.044241,2,4,1092,115,187,1193,1
1,Q2023398782,M2536956560,0.000922,1.546644,-0.438017,-0.274834,-1.404298,1.532982,-0.201699,1.690382,...,-0.074245,-0.028820,-0.044241,2,0,1092,115,368,450,1
2,Q4151338694,M3294926344,0.072103,0.314748,-0.438017,-0.274834,0.712100,-0.652323,-0.201699,-0.591582,...,-0.074245,-0.028820,-0.044241,0,4,1092,115,189,384,1
3,Q3271436624,M3744310794,0.660872,-1.621091,-0.438017,-0.274834,-1.404298,-0.652323,-0.201699,-0.591582,...,-0.055711,-0.028304,-0.036537,1,3,1092,115,173,899,0
4,Q3314287018,M1349051752,0.037847,1.018691,-0.438017,-0.274834,-1.404298,1.532982,-0.201699,-0.591582,...,-0.074245,-0.028820,-0.044241,0,4,350,170,315,683,1
5,Q4214103875,M2007129506,0.026668,-0.037222,-0.438017,-0.274834,-1.404298,1.532982,-0.201699,1.690382,...,-0.074245,-0.028820,-0.044241,1,0,1092,115,175,657,1
6,Q1421177878,M3927950819,0.093314,0.138763,-0.438017,-0.274834,-1.404298,1.532982,4.957871,-0.591582,...,-0.074245,-0.028820,-0.044241,0,0,532,170,187,709,1
7,Q3598252818,M2871943120,0.044356,-0.741162,-0.438017,-0.274834,-1.404298,1.532982,-0.201699,1.690382,...,-0.074245,-0.028820,-0.044241,1,4,1486,170,368,657,1
8,Q568518135,M998566127,0.252524,0.842704,1.313413,-0.274834,0.712100,-0.652323,-0.201699,-0.591582,...,-0.074245,-0.028820,-0.044241,2,2,1092,115,258,657,1
9,Q2242868437,M1307039867,0.042371,-0.389192,-0.438017,-0.274834,0.712100,-0.652323,-0.201699,-0.591582,...,-0.074245,-0.028820,-0.044241,0,0,1092,115,118,657,1


In [29]:
invite_info_evaluate = pd.read_csv('invite_info_evaluate_1_0926.txt', 
                                   header=None, sep='\t')
invite_info_evaluate.columns = ['q_id1', 'u_id1', 'invite_time']

In [30]:
result_out = pd.concat([result[['q_id', 'u_id', 'do_answer']], invite_info_evaluate],
                       axis=1)
len(result_out)

1141683

In [31]:
result_out[['q_id', 'u_id', 'invite_time', 
        'do_answer']].to_csv(RESULT_DIR_OUT, index=False, header=False, sep='\t')  # 导出数据

'ok'

'ok'